In [2]:
from bert_model import BertModel
from train_tempobert import ModelArguments
from transformers import AutoModelForMaskedLM, pipeline

In [3]:
import test_bert
import hf_utils

In [4]:
tester = test_bert.Tester('temp_att_model_semeval_eng', preload=True)

In [5]:
fill_mask_pipeline = tester.fill_mask_pipelines[0]

In [6]:
fill_mask_pipeline.tokenizer

PreTrainedTokenizerFast(name_or_path='temp_att_model_semeval_eng', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [7]:
sentence = "today it is the year of 1950."
time_tokens = [f"<{time}>" for time in fill_mask_pipeline.model.config.times]
time_tokens

['<1>', '<2>']

In [9]:
fill_mask_pipeline.tokenizer.tokenize(sentence)

ValueError: batch_time_id is missing

In [20]:
fill_mask_pipeline.model

TempoBertForMaskedLM(
  (cls): BertOnlyMLMHead(
    (predictions): BertLMPredictionHead(
      (transform): BertPredictionHeadTransform(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      )
      (decoder): Linear(in_features=768, out_features=30524, bias=True)
    )
  )
  (bert): TempoBertModel(
    (embeddings): TempoBertEmbeddings(
      (word_embeddings): Embedding(30524, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (time_embeddings): Embedding(4, 768)
    )
    (encoder): BertEncoderWithTemporalAttention(
      (layer): ModuleList(
        (0): BertLayerWithTemporalAttention(
          (attention): TemporalAttention(
            (self): TemporalSelfAttention(
              (query): Linea

In [15]:
sentence_extended = "[MASK] " + sentence
fill_result = fill_mask_pipeline(sentence_extended, targets=time_tokens)

ValueError: batch_time_id is missing

In [ ]:
model_name_or_path = 'temp_att_model_semeval_eng'
model_args = ModelArguments(model_name_or_path=model_name_or_path)

In [ ]:
model_args

In [ ]:
config_kwargs = {}
model, tokenizer = hf_utils.load_pretrained_model(
    model_args,
    AutoModelForMaskedLM,
    expect_times_in_model=False,
    revision='latest',
    **config_kwargs,
)

In [ ]:
model = test_bert.load_model('temp_att_model_semeval_eng',  expect_times_in_model=False)

In [ ]:
from transformers.tokenization_utils_base import get_fast_tokenizer_file

In [ ]:
get_fast_tokenizer_file?

In [ ]:
import transformers
transformers.__version__